# 1. Título - Indicadores sobre Ensino Superior

# 2. Membros
## 2020089879 - Chrystian Melo
## 2022043787 - Fernando Coutinho
## 2022037507 - Murilo Ribeiro
## 2020006914 - Nauan Barbosa

# 3. Descrição dos dados (qual a URL? qual o domínio? como os dados foram processados?)

[Dados do MEC](https://dados.gov.br/dados/conjuntos-dados/indicadores-sobre-ensino-superior) sobre Ensino Superior

## Processamento dos dados
.csv -> .db

# 4. Diagrama ER

(pendente)

# 5. Diagrama relacional

## https://docs.google.com/document/d/1r1sGPKGszgGQEFryLsbwUbDDY_2dZFVkm-qaL7ASRF8/edit?usp=sharing

# 6. Consultas

## Preparação do ambiente

In [1]:
import io
import pandas as pd
import json
import sqlite3
import json

from google.colab import drive
drive.flush_and_unmount()
drive.mount('/content/drive')

graduacao =  pd.read_csv("/content/drive/MyDrive/Colab Notebooks/IBD/tp2/Graduacao.csv")
especializacao = pd.read_csv("/content/drive/MyDrive/Colab Notebooks/IBD/tp2/Especializacao.csv")

conn = sqlite3.connect('mecData.db')

graduacao.to_sql('Graduacao', conn, if_exists='replace', index=False)
especializacao.to_sql('Especializacao', conn, if_exists='replace', index=False)

cursor = conn.cursor()

Drive not mounted, so nothing to flush and unmount.
Mounted at /content/drive


## Normalização 1

In [2]:
query = """
SELECT DISTINCT
  CODIGO_AREA_OCDE_CINE AS CODIGO_AREA_CONHECIMENTO, AREA_OCDE_CINE AS NOME_AREA_CONHECIMENTO
FROM
  graduacao
GROUP BY CODIGO_AREA_CONHECIMENTO
UNION
SELECT DISTINCT
  CODIGO_OCDE_CINE AS CODIGO_AREA_CONHECIMENTO, OCDE_CINE AS NOME_AREA_CONHECIMENTO
FROM
  especializacao
GROUP BY CODIGO_AREA_CONHECIMENTO
"""
df = pd.read_sql_query(query, conn)
df.to_sql('Tematica', conn, if_exists='replace', index=False)

cursor.execute("ALTER TABLE graduacao DROP COLUMN AREA_OCDE_CINE;")
cursor.execute("ALTER TABLE graduacao DROP COLUMN AREA_OCDE;") #coluna com dado duplicado

cursor.execute("ALTER TABLE especializacao DROP COLUMN OCDE_CINE;")

conn.commit()

print (df)

    CODIGO_AREA_CONHECIMENTO                       NOME_AREA_CONHECIMENTO
0                       None                                         None
1                        0.0                            Programas básicos
2                        1.0                                     Educação
3                        2.0                          Artes e humanidades
4                        3.0   Ciências sociais, comunicação e informação
..                       ...                                          ...
431                       95  Conservação e restauro de material cultural
432                       96                                     História
433                       97                                    Filosofia
434                       98    Línguas/literaturas estrangeiras modernas
435                       99         Redação criativa / expressão escrita

[436 rows x 2 columns]


In [4]:
query = """
SELECT DISTINCT
  CODIGO_MUNICIPIO, MUNICIPIO AS NOME_MUNICIPIO, UF, REGIAO
FROM
  graduacao
UNION
SELECT DISTINCT
  CODIGO_MUNICIPIO, MUNICIPIO AS NOME_MUNICIPIO, UF, REGIAO
FROM
  especializacao
"""

df = pd.read_sql_query(query, conn)
df.to_sql('Municipio', conn, if_exists='replace', index=False)

cursor.execute("ALTER TABLE graduacao DROP COLUMN MUNICIPIO;")
cursor.execute("ALTER TABLE graduacao DROP COLUMN UF;")
cursor.execute("ALTER TABLE graduacao DROP COLUMN REGIAO;")

cursor.execute("ALTER TABLE especializacao DROP COLUMN MUNICIPIO;")
cursor.execute("ALTER TABLE especializacao DROP COLUMN UF;")
cursor.execute("ALTER TABLE especializacao DROP COLUMN REGIAO;")

conn.commit()

print (df)

      CODIGO_MUNICIPIO         NOME_MUNICIPIO  UF             REGIAO
0              1100015  Alta Floresta D´Oeste  RO              NORTE
1              1100023              Ariquemes  RO              NORTE
2              1100049                 Cacoal  RO              NORTE
3              1100056             Cerejeiras  RO              NORTE
4              1100064      Colorado do Oeste  RO              NORTE
...                ...                    ...  ..                ...
3785           5221908                 Varjão  GO       CENTRO-OESTE
3786           5222005             Vianópolis  GO       CENTRO-OESTE
3787           5222054         Vicentinópolis  GO       CENTRO-OESTE
3788           5300108               Brasília  DF       CENTRO-OESTE
3789           9000001   Ignorado ou exterior  ZZ  IGNORADO/EXTERIOR

[3790 rows x 4 columns]


In [6]:
query = """
SELECT DISTINCT
  CODIGO_IES AS CODIGO_INSTITUICAO, NOME_IES AS NOME_INSTITUICAO
FROM
  graduacao
UNION
SELECT DISTINCT
  CODIGO_IES AS CODIGO_INSTITUICAO, NOME_IES AS NOME_INSTITUICAO
FROM
  especializacao
"""

df = pd.read_sql_query(query, conn)
df.to_sql('Instituicao', conn, if_exists='replace', index=False)

cursor.execute("ALTER TABLE graduacao DROP COLUMN NOME_IES;")
cursor.execute("ALTER TABLE graduacao DROP COLUMN CATEGORIA_ADMINISTRATIVA;")
cursor.execute("ALTER TABLE graduacao DROP COLUMN ORGANIZACAO_ACADEMICA;")

cursor.execute("ALTER TABLE especializacao DROP COLUMN NOME_IES;")

conn.commit()

print (df)

      CODIGO_INSTITUICAO                                   NOME_INSTITUICAO
0                      1                UNIVERSIDADE FEDERAL DE MATO GROSSO
1                      2                           UNIVERSIDADE DE BRASÍLIA
2                      3                    UNIVERSIDADE FEDERAL DE SERGIPE
3                      4                   UNIVERSIDADE FEDERAL DO AMAZONAS
4                      5                      UNIVERSIDADE FEDERAL DO PIAUÍ
...                  ...                                                ...
3874               27076  Escola de Saúde Pública do Estado de Minas Gerais
3875               27103                   UNIVERSIDADE DO DISTRITO FEDERAL
3876               29079      Centro de Estudos e Aperfeiçoamento Funcional
3877               29208  FUNDAÇÃO ESCOLA SUPERIOR DO MINISTÉRIO PÚBLICO...
3878               29224  ESCOLA DA DEFENSORIA PÚBLICA DO ESTADO DE SÃO ...

[3879 rows x 2 columns]


In [8]:
query = """
SELECT DISTINCT
  CODIGO_IES AS CODIGO_INSTITUICAO, CODIGO_MUNICIPIO
FROM
  graduacao
UNION
SELECT DISTINCT
  CODIGO_IES AS CODIGO_INSTITUICAO, CODIGO_MUNICIPIO
FROM
  especializacao
"""

df = pd.read_sql_query(query, conn)
df.to_sql('Local_Instituicao', conn, if_exists='replace', index=False)

cursor.execute("ALTER TABLE graduacao DROP COLUMN CODIGO_MUNICIPIO;")
cursor.execute("ALTER TABLE especializacao DROP COLUMN CODIGO_MUNICIPIO;")

conn.commit()

print (df)

       CODIGO_INSTITUICAO  CODIGO_MUNICIPIO
0                       1           5100201
1                       1           5100300
2                       1           5101308
3                       1           5101407
4                       1           5101704
...                   ...               ...
37921               27076           3106200
37922               27103           5300108
37923               29079           5103403
37924               29208           5103403
37925               29224           3550308

[37926 rows x 2 columns]


In [9]:
cursor.execute("ALTER TABLE Graduacao RENAME COLUMN CODIGO_IES TO COD_INSTITUICAO;")

cursor.execute("ALTER TABLE Especializacao RENAME COLUMN CODIGO_IES TO COD_INSTITUICAO;")

cursor.execute("ALTER TABLE Especializacao RENAME COLUMN CODIGO_OCDE_CINE TO COD_AREA_CONHECIMENTO;")

cursor.execute("ALTER TABLE Graduacao RENAME COLUMN CODIGO_AREA_OCDE_CINE TO COD_AREA_CONHECIMENTO;")

cursor.execute("ALTER TABLE Instituicao RENAME COLUMN CODIGO_INSTITUICAO TO COD_INSTITUICAO;")

conn.commit()

## Resultado final

In [12]:
# Consulta para obter os nomes das tabelas no banco de dados
cursor.execute("SELECT name FROM sqlite_master WHERE type='table';")
tables = cursor.fetchall()

# Exibir informações sobre as tabelas
print("Informações das tabelas no banco de dados:")
for table in tables:
    table_name = table[0]
    print(f"\nNome da Tabela: {table_name}")

    # Consulta para obter o número de linhas na tabela
    cursor.execute(f"SELECT COUNT(*) FROM {table_name};")
    rows_count = cursor.fetchone()[0]
    print(f"Número de Linhas: {rows_count}")

    # Consulta para obter os nomes e tipos de dados das colunas da tabela
    cursor.execute(f"PRAGMA table_info({table_name});")
    columns = cursor.fetchall()
    print("Colunas, Tipos de Dados e Chaves:")
    for column in columns:
        col_name = column[1]
        col_type = column[2]
        is_primary_key = column[5]  # Coluna 5 indica se é chave primária (1 = sim, 0 = não)
        primary_key_info = " (Primary-Key)" if is_primary_key else "(DATA)"
        print(f"  Nome da Coluna: {col_name}, Tipo de Dado: {col_type}{primary_key_info}")

    # Consulta para obter as chaves estrangeiras da tabela
    cursor.execute(f"PRAGMA foreign_key_list({table_name});")
    foreign_keys = cursor.fetchall()
    if foreign_keys:
        print("Chaves Estrangeiras:")
        for fk in foreign_keys:
            fk_column = fk[3]  # Coluna local (chave estrangeira)
            referenced_table = fk[2]  # Tabela referenciada
            referenced_column = fk[4]  # Coluna referenciada
            print(f"  Coluna: {fk_column} -> Tabela Referenciada: {referenced_table}, Coluna Referenciada: {referenced_column}")
    else:
        print("  Não há chaves estrangeiras.")


Informações das tabelas no banco de dados:

Nome da Tabela: Graduacao
Número de Linhas: 902676
Colunas, Tipos de Dados e Chaves:
  Nome da Coluna: COD_INSTITUICAO, Tipo de Dado: INTEGER(DATA)
  Nome da Coluna: CODIGO_CURSO, Tipo de Dado: INTEGER(DATA)
  Nome da Coluna: NOME_CURSO, Tipo de Dado: TEXT(DATA)
  Nome da Coluna: GRAU, Tipo de Dado: TEXT(DATA)
  Nome da Coluna: MODALIDADE, Tipo de Dado: TEXT(DATA)
  Nome da Coluna: SITUACAO_CURSO, Tipo de Dado: TEXT(DATA)
  Nome da Coluna: QT_VAGAS_AUTORIZADAS, Tipo de Dado: INTEGER(DATA)
  Nome da Coluna: CARGA_HORARIA, Tipo de Dado: INTEGER(DATA)
  Nome da Coluna: COD_AREA_CONHECIMENTO, Tipo de Dado: TEXT(DATA)
  Não há chaves estrangeiras.

Nome da Tabela: Especializacao
Número de Linhas: 677787
Colunas, Tipos de Dados e Chaves:
  Nome da Coluna: COD_INSTITUICAO, Tipo de Dado: INTEGER(DATA)
  Nome da Coluna: COD_DA_ESPECIALIZACAO, Tipo de Dado: INTEGER(DATA)
  Nome da Coluna: NOME_ESPECIALIZACAO, Tipo de Dado: TEXT(DATA)
  Nome da Coluna: 

## Normalização 2

In [19]:
# Criar a tabela Graduacao com as chaves primária e estrangeiras
cursor.execute('''
    CREATE TABLE IF NOT EXISTS Graduacao_new (
        COD_INSTITUICAO INTEGER,
        CODIGO_CURSO INTEGER,
        NOME_CURSO TEXT,
        GRAU TEXT,
        MODALIDADE TEXT,
        SITUACAO_CURSO TEXT,
        QT_VAGAS_AUTORIZADAS INTEGER,
        CARGA_HORARIA INTEGER,
        COD_AREA_CONHECIMENTO TEXT,
        PRIMARY KEY (CODIGO_CURSO),
        FOREIGN KEY (COD_INSTITUICAO) REFERENCES Instituicao(COD_INSTITUICAO),
        FOREIGN KEY (COD_AREA_CONHECIMENTO) REFERENCES Tematica(CODIGO_AREA_CONHECIMENTO)
    )
''')

# Copiar dados da tabela Graduacao, IGNORANDO DUPLICATAS
cursor.execute('''
    INSERT INTO Graduacao_new
    SELECT DISTINCT COD_INSTITUICAO, CODIGO_CURSO, NOME_CURSO, GRAU, MODALIDADE,
           SITUACAO_CURSO, QT_VAGAS_AUTORIZADAS, CARGA_HORARIA, COD_AREA_CONHECIMENTO
    FROM Graduacao
''')

conn.commit()

IntegrityError: UNIQUE constraint failed: Graduacao_new.CODIGO_CURSO

In [21]:
# Criar a tabela Especializacao com chaves primária e estrangeiras
cursor.execute('''
    CREATE TABLE IF NOT EXISTS Especializacao_new (
        COD_INSTITUICAO INTEGER,
        COD_DA_ESPECIALIZACAO INTEGER,
        NOME_ESPECIALIZACAO TEXT,
        COD_AREA_CONHECIMENTO REAL,
        CARGA_HORARIA INTEGER,
        DURACAO_MESES INTEGER,
        MODALIDADE TEXT,
        VAGAS INTEGER,
        SITUACAO TEXT,
        PRIMARY KEY (COD_DA_ESPECIALIZACAO),
        FOREIGN KEY (COD_INSTITUICAO) REFERENCES Instituicao(COD_INSTITUICAO),
        FOREIGN KEY (COD_AREA_CONHECIMENTO) REFERENCES Tematica(CODIGO_AREA_CONHECIMENTO)
    )
''')

# Copiar dados da tabela Especializacao, IGNORANDO DUPLICATAS
cursor.execute('''
    INSERT INTO Especializacao_new
    SELECT DISTINCT COD_INSTITUICAO, COD_DA_ESPECIALIZACAO, NOME_ESPECIALIZACAO,
           COD_AREA_CONHECIMENTO, CARGA_HORARIA, DURACAO_MESES, MODALIDADE, VAGAS, SITUACAO
    FROM Especializacao
''')
conn.commit()


In [23]:
# Criar a tabela Tematica com chave primária
cursor.execute('''
    CREATE TABLE IF NOT EXISTS Tematica_new (
        CODIGO_AREA_CONHECIMENTO TEXT PRIMARY KEY,
        NOME_AREA_CONHECIMENTO TEXT
    )
''')

# Copiar dados da tabela Tematica
cursor.execute('''
    INSERT INTO Tematica_new
    SELECT DISTINCT CODIGO_AREA_CONHECIMENTO, NOME_AREA_CONHECIMENTO
    FROM Tematica
''')
conn.commit()


In [24]:
# Criar a tabela Municipio com chave primária
cursor.execute('''
    CREATE TABLE IF NOT EXISTS Municipio_new (
        CODIGO_MUNICIPIO INTEGER PRIMARY KEY,
        NOME_MUNICIPIO TEXT,
        UF TEXT,
        REGIAO TEXT
    )
''')

# Copiar dados da tabela Municipio, IGNORANDO DUPLICATAS
cursor.execute('''
    INSERT INTO Municipio_new
    SELECT DISTINCT CODIGO_MUNICIPIO, NOME_MUNICIPIO, UF, REGIAO
    FROM Municipio
''')
conn.commit()


In [30]:
# Criar a tabela Instituicao com chave primária
cursor.execute('''
    CREATE TABLE IF NOT EXISTS Instituicao_new (
        COD_INSTITUICAO INTEGER PRIMARY KEY,
        NOME_INSTITUICAO TEXT
    )
''')

# Copiar dados da tabela Instituicao, mantendo um único registro para cada COD_INSTITUICAO
cursor.execute('''
    INSERT INTO Instituicao_new
    SELECT COD_INSTITUICAO, MAX(NOME_INSTITUICAO)
    FROM Instituicao
    GROUP BY COD_INSTITUICAO
''')

conn.commit()

In [26]:
# Criar a tabela Local_Instituicao com chaves primária e estrangeiras
cursor.execute('''
    CREATE TABLE IF NOT EXISTS Local_Instituicao_new (
        CODIGO_INSTITUICAO INTEGER,
        CODIGO_MUNICIPIO INTEGER,
        PRIMARY KEY (CODIGO_INSTITUICAO, CODIGO_MUNICIPIO),
        FOREIGN KEY (CODIGO_INSTITUICAO) REFERENCES Instituicao(COD_INSTITUICAO),
        FOREIGN KEY (CODIGO_MUNICIPIO) REFERENCES Municipio(CODIGO_MUNICIPIO)
    )
''')

# Copiar dados da tabela Local_Instituicao, IGNORANDO DUPLICATAS
cursor.execute('''
    INSERT INTO Local_Instituicao_new
    SELECT DISTINCT CODIGO_INSTITUICAO, CODIGO_MUNICIPIO
    FROM Local_Instituicao
''')

conn.commit()

In [32]:
# Excluir as tabelas antigas
cursor.execute('DROP TABLE IF EXISTS Graduacao')
cursor.execute('DROP TABLE IF EXISTS Especializacao')
cursor.execute('DROP TABLE IF EXISTS Tematica')
cursor.execute('DROP TABLE IF EXISTS Municipio')
cursor.execute('DROP TABLE IF EXISTS Instituicao')
cursor.execute('DROP TABLE IF EXISTS Local_Instituicao')

# Renomear as novas tabelas para os nomes originais
cursor.execute('ALTER TABLE Graduacao_new RENAME TO Graduacao')
cursor.execute('ALTER TABLE Especializacao_new RENAME TO Especializacao')
cursor.execute('ALTER TABLE Tematica_new RENAME TO Tematica')
cursor.execute('ALTER TABLE Municipio_new RENAME TO Municipio')
cursor.execute('ALTER TABLE Instituicao_new RENAME TO Instituicao')
cursor.execute('ALTER TABLE Local_Instituicao_new RENAME TO Local_Instituicao')

# Commitar as mudanças
conn.commit()

## Resultado final

In [34]:
# Consulta para obter os nomes das tabelas no banco de dados
cursor.execute("SELECT name FROM sqlite_master WHERE type='table';")
tables = cursor.fetchall()

# Exibir informações sobre as tabelas
print("Informações das tabelas no banco de dados:")
for table in tables:
    table_name = table[0]
    print(f"\nNome da Tabela: {table_name}")

    # Consulta para obter o número de linhas na tabela
    cursor.execute(f"SELECT COUNT(*) FROM {table_name};")
    rows_count = cursor.fetchone()[0]
    print(f"Número de Linhas: {rows_count}")

    # Consulta para obter os nomes e tipos de dados das colunas da tabela
    cursor.execute(f"PRAGMA table_info({table_name});")
    columns = cursor.fetchall()
    print("Colunas, Tipos de Dados e Chaves:")
    for column in columns:
        col_name = column[1]
        col_type = column[2]
        is_primary_key = column[5]  # Coluna 5 indica se é chave primária (1 = sim, 0 = não)
        primary_key_info = " (Primary-Key)" if is_primary_key else "(DATA)"
        print(f"  Nome da Coluna: {col_name}, Tipo de Dado: {col_type}{primary_key_info}")

    # Consulta para obter as chaves estrangeiras da tabela
    cursor.execute(f"PRAGMA foreign_key_list({table_name});")
    foreign_keys = cursor.fetchall()
    if foreign_keys:
        print("Chaves Estrangeiras:")
        for fk in foreign_keys:
            fk_column = fk[3]  # Coluna local (chave estrangeira)
            referenced_table = fk[2]  # Tabela referenciada
            referenced_column = fk[4]  # Coluna referenciada
            print(f"  Coluna: {fk_column} -> Tabela Referenciada: {referenced_table}, Coluna Referenciada: {referenced_column}")
    else:
        print("  Não há chaves estrangeiras.")


Informações das tabelas no banco de dados:

Nome da Tabela: Graduacao
Número de Linhas: 86239
Colunas, Tipos de Dados e Chaves:
  Nome da Coluna: COD_INSTITUICAO, Tipo de Dado: INTEGER(DATA)
  Nome da Coluna: CODIGO_CURSO, Tipo de Dado: INTEGER (Primary-Key)
  Nome da Coluna: NOME_CURSO, Tipo de Dado: TEXT(DATA)
  Nome da Coluna: GRAU, Tipo de Dado: TEXT(DATA)
  Nome da Coluna: MODALIDADE, Tipo de Dado: TEXT(DATA)
  Nome da Coluna: SITUACAO_CURSO, Tipo de Dado: TEXT(DATA)
  Nome da Coluna: QT_VAGAS_AUTORIZADAS, Tipo de Dado: INTEGER(DATA)
  Nome da Coluna: CARGA_HORARIA, Tipo de Dado: INTEGER(DATA)
  Nome da Coluna: COD_AREA_CONHECIMENTO, Tipo de Dado: TEXT(DATA)
Chaves Estrangeiras:
  Coluna: COD_AREA_CONHECIMENTO -> Tabela Referenciada: Tematica, Coluna Referenciada: CODIGO_AREA_CONHECIMENTO
  Coluna: COD_INSTITUICAO -> Tabela Referenciada: Instituicao, Coluna Referenciada: COD_INSTITUICAO

Nome da Tabela: Especializacao
Número de Linhas: 233119
Colunas, Tipos de Dados e Chaves:
  No

## 6.1 Duas consultas envolvendo seleção e projeção

### 6.1.1 Consulta 1
//selecao de cursos de graduacao a distancia

In [35]:
query = """
SELECT DISTINCT
  NOME_CURSO
FROM
  GRADUACAO
WHERE
  GRADUACAO.MODALIDADE = 'Educação a Distância'
"""

df = pd.read_sql_query(query, conn)
df

,NOME_CURSO
0,PEDAGOGIA
1,TECNOLOGIA DA INFORMAÇÃO
2,ADMINISTRAÇÃO
3,MAGISTÉRIO DAS SÉRIES INICIAIS DO ENSINO FUNDA...
4,MATEMÁTICA
...,...
630,DESENVOLVIMENTO DE APLICATIVOS PARA DISPOSITIV...
631,GESTÃO DE CLÍNICAS E CONSULTÓRIOS
632,APICULTURA E MELIPONICULTURA
633,PROGRAMA ESPECIAL DE FORMAÇÃO PEDAGÓGICA DE DO...


### 6.1.2 Consulta 2
//selecao de cursos de graduacao com carga horaria maior ou igual a 4000 horas

In [36]:
query = """
SELECT DISTINCT
  NOME_CURSO
FROM
  GRADUACAO
WHERE
  CARGA_HORARIA >= 4000
"""

df = pd.read_sql_query(query, conn)
df

,NOME_CURSO
0,ENGENHARIA CIVIL
1,AGRONOMIA
2,ENGENHARIA FLORESTAL
3,MEDICINA
4,ENFERMAGEM
...,...
389,ABI - PSICOLOGIA
390,ABI - ENFERMAGEM
391,EDUCAçãO DO CAMPO - FíSICA E BIOLOGIA
392,ABI - ENGENHARIA DE MINAS


## 6.2 Três consultas envolvendo junção de duas relações

### 6.2.1 Consulta 3
//instituicoes de ensino que ofertam tanto cursos de graduacao quanto de especializacao


In [37]:
query = """
SELECT DISTINCT
  NOME_INSTITUICAO
FROM
  GRADUACAO, ESPECIALIZACAO, INSTITUICAO
WHERE
  GRADUACAO.COD_INSTITUICAO = ESPECIALIZACAO.COD_INSTITUICAO
  AND GRADUACAO.COD_INSTITUICAO = INSTITUICAO.COD_INSTITUICAO;
"""

df = pd.read_sql_query(query, conn)
df

,NOME_INSTITUICAO
0,UNIVERSIDADE FEDERAL DE MATO GROSSO
1,Universidade Federal de Rondonópolis
2,UNIVERSIDADE DE BRASÍLIA
3,UNIVERSIDADE FEDERAL DE SERGIPE
4,UNIVERSIDADE FEDERAL DO AMAZONAS
...,...
2337,FACULDADE TEOLÓGICA EVANGÉLICA DO RIO DE JANEIRO
2338,FACULDADE DE CIÊNCIAS MÉDICAS E JURÍDICA
2339,INSTITUTO TOCANTINENSE PRESIDENTE ANTONIO CARLOS
2340,CENTRO SUPERIOR DE ESTUDOS JURÍDICOS CARLOS DR...


### 6.2.2 Consulta 4
//cursos de graduacao ofertados no municipio de Belo Horizonte

In [38]:
query = """
SELECT DISTINCT
  NOME_CURSO
FROM
  GRADUACAO, INSTITUICAO, LOCAL_INSTITUICAO, MUNICIPIO
WHERE
  GRADUACAO.COD_INSTITUICAO = LOCAL_INSTITUICAO.CODIGO_INSTITUICAO AND LOCAL_INSTITUICAO.CODIGO_MUNICIPIO = MUNICIPIO.CODIGO_MUNICIPIO
  AND MUNICIPIO.NOME_MUNICIPIO = 'Belo Horizonte'

"""

df = pd.read_sql_query(query, conn)
df

,NOME_CURSO
0,ABI - EDUCAÇÃO FÍSICA
1,ADMINISTRAÇÃO
2,ALIMENTOS
3,ANÁLISE E DESENVOLVIMENTO DE SISTEMAS
4,ARQUITETURA E URBANISMO
...,...
860,FORMAÇÃO PARA YOUTUBER
861,FITOTERAPIA
862,PEDAGOGIA - EDUCAÇÃO INFANTIL E ANOS INICIAIS ...
863,CIÊNCIAS MILITARES


### 6.2.3 Consulta 5
//cursos de especializacao ofertados no municipio de Belo Horizonte

In [39]:
query = """
SELECT DISTINCT
  NOME_ESPECIALIZACAO
FROM
  ESPECIALIZACAO, INSTITUICAO, LOCAL_INSTITUICAO, MUNICIPIO
WHERE
  ESPECIALIZACAO.COD_INSTITUICAO = LOCAL_INSTITUICAO.CODIGO_INSTITUICAO AND LOCAL_INSTITUICAO.CODIGO_MUNICIPIO = MUNICIPIO.CODIGO_MUNICIPIO
  AND MUNICIPIO.NOME_MUNICIPIO = 'Belo Horizonte'
"""

df = pd.read_sql_query(query, conn)
df

,NOME_ESPECIALIZACAO
0,Aprendizagem e Neurociência no Contexto Escolar
1,Arquitetura Comercial
2,Audiovisual e Convergência de Mídias
3,Automação Industrial
4,Automação e Indústria 4.0
...,...
52476,Curso de Pós-Graduação em Gestão Estratégica e...
52477,Curso de Pós-Graduação em Gestão e Direito Amb...
52478,Master in data and Decision Science
52479,Product Growth


## 6.3 Três consultas envolvendo junção de três ou mais relações


### 6.3.1 Consulta 6
//instituicoes de ensino localizadas em Belo Horizonte que ofertam tanto cursos de graduacao quanto de especializacao

In [40]:
query = """
SELECT DISTINCT
  NOME_INSTITUICAO
FROM
  GRADUACAO, ESPECIALIZACAO, INSTITUICAO, LOCAL_INSTITUICAO, MUNICIPIO
WHERE
  GRADUACAO.COD_INSTITUICAO = LOCAL_INSTITUICAO.CODIGO_INSTITUICAO AND ESPECIALIZACAO.COD_INSTITUICAO = LOCAL_INSTITUICAO.CODIGO_INSTITUICAO AND LOCAL_INSTITUICAO.CODIGO_MUNICIPIO = MUNICIPIO.CODIGO_MUNICIPIO
  AND MUNICIPIO.NOME_MUNICIPIO = 'Belo Horizonte' AND LOCAL_INSTITUICAO.CODIGO_INSTITUICAO = INSTITUICAO.COD_INSTITUICAO
"""

df = pd.read_sql_query(query, conn)
df

,NOME_INSTITUICAO
0,UNIVERSIDADE DO VALE DO RIO DOS SINOS
1,PONTIFÍCIA UNIVERSIDADE CATÓLICA DO RIO GRANDE...
2,UNIVERSIDADE PRESBITERIANA MACKENZIE
3,Centro Universitário Vale do Rio Verde
4,UNIVERSIDADE DO ESTADO DE MINAS GERAIS
...,...
275,Instituto Brasil de Ensino Superior
276,Faculdade Brasil Empreender
277,SIRIUS ESCOLA SUPERIOR DE TECNOLOGIA
278,ACADEMIA DE POLÍCIA MILITAR DE MINAS GERAIS


### 6.3.2 Consulta 7
//municipios que possuem instituicoes de ensino tanto de graduacao quanto de especializacao


In [41]:
query = """
SELECT DISTINCT
  NOME_MUNICIPIO
FROM
  GRADUACAO, ESPECIALIZACAO, LOCAL_INSTITUICAO, MUNICIPIO
WHERE
  GRADUACAO.COD_INSTITUICAO = LOCAL_INSTITUICAO.CODIGO_INSTITUICAO AND ESPECIALIZACAO.COD_INSTITUICAO = LOCAL_INSTITUICAO.CODIGO_INSTITUICAO AND LOCAL_INSTITUICAO.CODIGO_MUNICIPIO = MUNICIPIO.CODIGO_MUNICIPIO
"""

df = pd.read_sql_query(query, conn)
df

,NOME_MUNICIPIO
0,Água Boa
1,Alto Araguaia
2,Arenápolis
3,Aripuanã
4,Barra do Bugres
...,...
3605,Canapi
3606,Minador do Negrão
3607,Jacuí
3608,Tocantins


### 6.3.3 Consulta 8
//curso de especializacao e de graduacao ofertados no estado de Minas Gerais


In [ ]:
query = """
SELECT DISTINCT
  NOME_CURSO
FROM
  GRADUACAO, ESPECIALIZACAO, LOCAL_INSTITUICAO, MUNICIPIO
WHERE
  (GRADUACAO.COD_INSTITUICAO = LOCAL_INSTITUICAO.CODIGO_INSTITUICAO OR ESPECIALIZACAO.COD_INSTITUICAO = LOCAL_INSTITUICAO.CODIGO_INSTITUICAO) AND LOCAL_INSTITUICAO.CODIGO_MUNICIPIO = MUNICIPIO.CODIGO_MUNICIPIO
  AND MUNICIPIO.UF = 'MG'
"""

df = pd.read_sql_query(query, conn)
df

## 6.4 Duas consultas envolvendo agregação sobre junção de duas ou mais relações

### 6.4.1 Consulta 9
//vagas totais de especializacao e graduacao em Minas Gerais

In [ ]:
query = """
SELECT DISTINCT
  QT_VAGAS_AUTORIZADAS
FROM
  GRADUACAO, ESPECIALIZACAO, LOCAL_INSTITUICAO
WHERE
  (GRADUACAO.COD_INSTITUICAO = LOCAL_INSTITUICAO.CODIGO_INSTITUICAO OR ESPECIALIZACAO.COD_INSTITUICAO = LOCAL_INSTITUICAO.CODIGO_INSTITUICAO) AND LOCAL_INSTITUICAO.CODIGO_MUNICIPIO = MUNICIPIO.CODIGO_MUNICIPIO
  AND MUNICIPIO.UF = 'MG'
GROUP BY
  GRADUACAO.QT_VAGAS_AUTORIZADAS, ESPECIALIZACAO.QT_VAGAS_AUTORIZADAS
"""

df = pd.read_sql_query(query, conn)
df

### 6.4.2 Consulta 10
//instituicoes ordenadas por numero de vagas de graduacao e especializacao

In [ ]:
query = """
SELECT DISTINCT
  NOME_INSTITUICAO
FROM
  GRADUACAO, ESPECIALIZACAO, INSTITUICAO
WHERE
  GRADUACAO.COD_INSTITUICAO = INSTITUICAO.COD_INSTITUICAO OR ESPECIALIZACAO.COD_INSTITUICAO = INSTITUICAO.COD_INSTITUICAO
ORDER BY
  GRADUACAO.QT_VAGAS_AUTORIZADAS, ESPECIALIZACAO.QT_VAGAS_AUTORIZADAS DESC
"""

df = pd.read_sql_query(query, conn)
df

# 7. Autoavaliação dos membros

## Chrystian Melo 10/10
## Fernando Coutinho  10/10
## Murillo Ribeiro  10/10
## Nauan Barbosa 10/10